In [1]:
import pandas as pd
import folium, requests, vincent
from utils import *
vincent.core.initialize_notebook()
pd.options.mode.chained_assignment = None

# Question 1

### Data loading

The following dataframe contains our initial data that we are interested in.

In [2]:
# We load the grant data from the csv file and keep only the columns we are interested in.
df = pd.read_csv('Data/P3_GrantExport.csv', sep=';')
df.rename(columns={'\ufeff"Project Number"': 'Project Number'}, inplace=True)
df_projected = df[['Project Number', 'University', 'Start Date', 'End Date', 'Approved Amount']]
df_projected.head()

,Project Number,University,Start Date,End Date,Approved Amount
0,1,Nicht zuteilbar - NA,01.10.1975,30.09.1976,11619.00
1,4,Université de Genève - GE,01.10.1975,30.09.1976,41022.00
2,5,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.03.1976,28.02.1985,79732.00
3,6,Universität Basel - BS,01.10.1975,30.09.1976,52627.00
4,7,"NPO (Biblioth., Museen, Verwalt.) - NPO",01.01.1976,30.04.1978,120042.00


The following dictionary provides all 26 Swiss cantons along with their corresponding abbreviations.

In [3]:
# load json file that contains canton names
json_data = pd.read_json('ch-cantons.topojson.json', typ='dataframe')
# extract canton id
cantonid = [item['id'] for item in json_data['objects']['cantons']['geometries']]
# extract canton name
cantonname =[item['properties']['name'] for item in json_data['objects']['cantons']['geometries']]
# build swiss_cantons dictionary
swiss_cantons = dict(zip(cantonid,cantonname))
#swiss_cantons

### Data filtering

All the universities that are registered as NPO and NA are filtered out. We cannot extract any useful information regarding their location in Switzerland and therefore they are not included in our analysis. 

In [4]:
df_filtered = df_projected[
    (df_projected['University'].str.endswith('- NPO') == False) & 
    (df_projected['University'].str.endswith('- NA') == False)]
df_filtered['Canton'] = ''

### Find the canton of each University - Method I

We apply the <code>extract_canton</code> function to the remaining records. This function extracts the canton of its university from its name.

**NOTE:** After this step, we are going to have some false canton entries. We fix those in the next steps.

In [5]:
df_filtered['Canton'] = df_filtered.apply(lambda row: extract_canton(row['University']), axis=1)
df_filtered['University'] = df_filtered.apply(lambda row: modify_university_name(row['University']), axis=1)
df_filtered.head()

,Project Number,University,Start Date,End Date,Approved Amount,Canton
1,4,Université de Genève,01.10.1975,30.09.1976,41022.00,GE
3,6,Universität Basel,01.10.1975,30.09.1976,52627.00,BS
5,8,Université de Fribourg,01.01.1976,31.12.1978,53009.00,FR
6,9,Université de Fribourg,01.01.1976,31.12.1976,25403.00,FR
7,10,Universität Zürich,01.10.1975,31.03.1977,47100.00,ZH


The resulting cantons are displayed here. Obsviously, we have some false records.

In [6]:
df_filtered['Canton'].unique()

array(['GE', 'BS', 'FR', 'ZH', 'LA', 'BE', 'WSL', 'NE', 'ETHZ', 'IHEID',
       'SG', 'FINST', 'FP', 'PHGR', 'EPFL', 'PHZFH', 'LU', 'SIK-ISEA',
       'SUPSI', 'HES-SO', 'RWS', 'PSI', 'PHSG', 'EAWAG', 'EMPA', 'PMOD',
       'BFH', 'CSEM', 'ASPIT', 'AORI', 'SIAF', 'FIBL', 'FMI', 'KSPSG',
       'AGS', 'EOC', 'IST', 'EHB', 'ZFH', 'USI', 'IKG', 'FHNW', 'HfH',
       'CREALP', 'IDIAP', 'PHBern', 'IUKB', 'SPF', 'HSLU', 'IRO', 'HEPL',
       'FHO', 'ISSKA', 'SIB', 'HEPBEJUNE', 'PHLU',
       'Forschungskommission SAGW', 'ISR', 'ASP', 'HEPFR', 'PHSZ', 'PHTG',
       'BITG', 'FHKD', 'FORS', 'PHVS', 'SHLR', 'PHZG', 'IRSOL', 'FUS',
       'PHSH', 'PHFHNW', 'STHB', 'FTL', 'FFHS'], dtype=object)

### Find the canton of each University - Method II

We detect that Vaud (VD) appears as LA. Moreover, we have some records with ETHZ and EPFL which belong to the canton of ZH and VD respectively. We fix those entries manually.

In [7]:
df_filtered.loc[(df_filtered['Canton'] == 'LA') | (df_filtered['Canton'] == 'EPFL'),'Canton'] = 'VD'
df_filtered.loc[(df_filtered['Canton'] == 'ETHZ'), 'Canton'] = 'ZH'
df_filtered.head(3)

,Project Number,University,Start Date,End Date,Approved Amount,Canton
1,4,Université de Genève,01.10.1975,30.09.1976,41022.00,GE
3,6,Universität Basel,01.10.1975,30.09.1976,52627.00,BS
5,8,Université de Fribourg,01.01.1976,31.12.1978,53009.00,FR


### Find the canton of each University - Method III


We filter rows with false canton id, i.e. rows whose canton abbreviation does not belong to the 26 swiss cantons. In next steps, we try to find the real canton ids for those rows using the university name. 

In [8]:
uni_wrong_canton = df_filtered[df_filtered['Canton'].isin(swiss_cantons.keys()) == False]
uni_wrong_canton[['University','Canton']].head()

,University,Canton
29,"Eidg. Forschungsanstalt für Wald,Schnee,Land",WSL
56,Inst. de Hautes Etudes Internat. et du Dév,IHEID
78,Inst. de Hautes Etudes Internat. et du Dév,IHEID
83,Weitere Institute,FINST
84,Weitere Institute,FINST


We use the python <code>geocoder</code> API provided by Google to find the location of universities. This is done by calling the <code>find_canton_from_google</code> function from the <code>utils</code> file using the university name. We extract the state information and check if it is one of the 26 swiss cantons. If so, we assign the right canton value for each university. Otherwise, we assign a NaN value to the canton attribute.

In [9]:
# dictionary: key = univ. name; value = canton of univ.
uni_to_cantons = {}
# find university names that are assigned an invalid canton
unassigned_universities = uni_wrong_canton.groupby(by='Canton')['University'].unique().tolist()
line_counter = 1
for uni in unassigned_universities:
    university_name = uni[0]
    # find canton from university name using the geocoder API
    university_canton = find_canton_from_google(university_name)
    # keep only universities that are in Switzerland
    if university_canton in swiss_cantons.keys():
        uni_to_cantons[university_name] = university_canton
    print(line_counter, '. ', university_name, '|', university_canton)
    line_counter += 1

1 .  Forschungsanstalten Agroscope | nan
2 .  AO Research Institute | Wien
3 .  Pädag. Hochschule Tessin (Teilschule SUPSI) | TI
4 .  Weitere Spitäler | nan
5 .  Berner Fachhochschule | nan
6 .  Biotechnologie Institut Thurgau | TG
7 .  Centre de rech. sur l'environnement alpin | nan
8 .  Swiss Center for Electronics and Microtech. | nan
9 .  Eidg. Anstalt für Wasserversorgung | nan
10 .  Eidg. Hochschulinstitut für Berufsbildung | nan
11 .  Eidg. Material und Prüfungsanstalt | nan
12 .  Ente Ospedaliero Cantonale | nan
13 .  Fernfachhochschule Schweiz (Mitglied SUPSI) | nan
14 .  Fachhochschule Kalaidos | nan
15 .  Fachhochschule Nordwestschweiz (ohne PH) | nan
16 .  Fachhochschule Ostschweiz | nan
17 .  Forschungsinstitut für biologischen Landbau | nan
18 .  Weitere Institute | nan
19 .  Friedrich Miescher Institute | nan
20 .  Schweizer Kompetenzzentrum Sozialwissensch. | nan
21 .  Firmen/Privatwirtschaft | nan
22 .  Facoltà di Teologia di Lugano | TI
23 .  Franklin University Switz

Now we count the remaining universities with unknown cantons. We still have some universities that do not have a valid entry for the canton attribute.

In [10]:
len(unassigned_universities) - len(uni_to_cantons)

47

The <code>uni_to_cantons</code> dictionary contains university names and their respective canton, as returned using the <code>geocoder</code> API. We use the <code>fill_unknown_cantons</code> function from the <code>utils</code> file. We iterate through the whole dataframe and check if the university name is contained in the <code>uni_to_cantons</code> dictionary. If so, we check that the respective canton id in the dictionary is indeed one of the 26 swiss cantons and set the right value to the canton attribute. Otherwise, the value becomes NaN (e.g. one university reports Roma as canton, which is not a valid one).

In [11]:
df_filtered['Canton'] = df_filtered.apply(lambda row : fill_unknown_cantons(swiss_cantons, uni_to_cantons, row['University'], row['Canton']), axis=1)

In [12]:
df_filtered.head()

,Project Number,University,Start Date,End Date,Approved Amount,Canton
1,4,Université de Genève,01.10.1975,30.09.1976,41022.00,GE
3,6,Universität Basel,01.10.1975,30.09.1976,52627.00,BS
5,8,Université de Fribourg,01.01.1976,31.12.1978,53009.00,FR
6,9,Université de Fribourg,01.01.1976,31.12.1976,25403.00,FR
7,10,Universität Zürich,01.10.1975,31.03.1977,47100.00,ZH


In [13]:
df_filtered.shape

(46920, 6)

Here, we see that the number of universities with unknown canton is further reduced.

In [14]:
unique_unis = df_filtered[['University', 'Canton']].drop_duplicates()
unassigned_uni = unique_unis[(unique_unis['Canton'].isnull())]
unassigned_uni.shape[0]

47

### Find the canton of each University - Method IV

Finally, we exploit the fact that the canton name may be embedded in the university name. Thus, we search for a substing in the university name that contains a canton name.

First, we build a list that contains all canton names (some cantons may be known with multiple names: e.g. Bern and Berne).

In [15]:
# some cantons have two names seperated with /
canton_names_list = [item.split('/') for item in swiss_cantons.values()]
# keep a list of all possible canton names
canton_names_list = [item for sublist in canton_names_list for item in sublist]

Now, we check if any canton name is embedded in the university name. If so, the university is assigned the respective canton (e.g.'Institut für Kulturforschung Graubünden': 'GR').

In [16]:
assigned_cantons = {}
for canton in canton_names_list:
    for university in unassigned_uni['University']:
        if(canton.lower() in university.lower()):
            k = [key for key, value in swiss_cantons.items() if canton in value]
            assigned_cantons[university] = k[0]
print(assigned_cantons)

{'Pädagogische Hochschule Wallis': 'VS', 'Pädagogische Hochschule Zug': 'ZG', 'Haute école pédagogique du canton de Vaud': 'VD', 'Berner Fachhochschule': 'BE', 'Haute école pédagogique fribourgeoise': 'FR'}


We iterate again through the whole dataframe to fix rows that contain university names whose canton is now known.

In [17]:
df_filtered['Canton'] = df_filtered.apply(lambda row : fill_unknown_cantons(swiss_cantons, assigned_cantons, row['University'], row['Canton']), axis=1)

From now on, we use the <code>uni_grants</code> dataframe

In [18]:
uni_grants = df_filtered

We through away universities that do not contain any valid value in the 'Approved Amount' field. Moreover, we drop universities with names 'Weitere Institute' and 'Weitere Spitäler' which stands for other institutions in German.

In [19]:
uni_grants = uni_grants[uni_grants['Approved Amount'] != 'data not included in P3']
uni_grants = uni_grants[uni_grants['University'] != 'Weitere Institute']
uni_grants = uni_grants[uni_grants['University'] != 'Weitere Spitäler']

Now, we calculate the percentage of rows that are contained in the mofidied dataframe and have a valid swiss canton in the 'Canton' attribute.

In [20]:
num_of_assigned_universities =uni_grants.shape[0] 
num_of_unassigned_universities = uni_grants['Canton'].isnull().sum()
accuracy = (1 - float(num_of_unassigned_universities) / num_of_assigned_universities) * 100
str_acc = "{:.2f}".format(accuracy)
print('Accuracy: ',str_acc,'%')

Accuracy:  92.62 %


We convert the 'Approved Amount' attribute to float.

In [21]:
uni_grants['Approved Amount'] = uni_grants['Approved Amount'].astype('float64')

We keep the unique university names and use the <code>geocoder</code> API to find their location. We will use the locations to put a marker on the swiss map for each university.

In [22]:
# keep unique univ. names
uni_names = uni_grants.dropna()['University'].unique()
# dictionary: key = univ. name; value = univ. location
uni_locs = {}
for uni_name in uni_names:
    location = get_university_location(uni_name)
    # print(uni_name,location)
    if location:
        uni_locs[uni_name] = location

Not all swiss cantons have a university. We create a new dataframe that contains the canton ids of such cantons and set the approved amount for grants to 0.0.

In [23]:
uni_grants.dropna(subset=['Canton'], inplace=True)
# find cantons with at least one university
cantons_with_uni = list(uni_grants['Canton'].unique())
# find canton without university
cantons_without_uni = pd.Series([item for item in swiss_cantons.keys() if item not in cantons_with_uni])
cantons_without_uni = cantons_without_uni.to_frame()
cantons_without_uni.columns = ['Canton']
# set approved amount to 0.0 for cantons without univ.
cantons_without_uni['Approved Amount'] = 0.0
cantons_without_uni

,Canton,Approved Amount
0,UR,0.0
1,GL,0.0
2,SO,0.0
3,JU,0.0
4,NW,0.0
5,BL,0.0
6,OW,0.0
7,AI,0.0
8,AR,0.0


Now, we append the newly created dataframe to the <code>uni_grants</code> dataframe. This action is necessary, in order for the swiss university map to be created properly.

In [24]:
uni_grants = uni_grants.append(cantons_without_uni)

We group by the canton and sum the approved amount of each university to find the total approved amount per canton. 

**NOTE:** We report the results in MCHF.

In [25]:
grants_per_canton = uni_grants.groupby(by='Canton')['Approved Amount'].sum()
grants_per_canton = grants_per_canton.to_frame()
grants_per_canton.reset_index(inplace=True)
grants_per_canton['Approved Amount'] = grants_per_canton['Approved Amount']/1000000

In [26]:
max_amount = grants_per_canton['Approved Amount'].max()
min_amount = grants_per_canton['Approved Amount'].min()

<font color='red'>**NOTE: The <code>legend_name</code> is not displayed in the map. The color palette is used to visualize the amount of approved grant per canton. The deeper the green hue, the higher the approved amount of money (MCHF).**</font>

In [27]:
canton_geo = r'ch-cantons.topojson.json'
canton_data = grants_per_canton

swiss_map = folium.Map(location=[46.762579, 7.927242], zoom_start=8)
swiss_map.choropleth(geo_path= canton_geo, data=canton_data,
                     key_on='feature.id',
                     threshold_scale=list(range(int(min_amount), int(max_amount),700)),
                     columns=['Canton', 'Approved Amount'],
                     fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                     topojson='objects.cantons',
                     legend_name='Approved Amount (MCHF)')

for key,value in uni_locs.items():
    folium.Marker(value, popup=key).add_to(swiss_map)

swiss_map.save('Maps/swiss-map.html')
swiss_map

In [5]:
html="""
    <h1> This is a big popup</h1><br>
    With a few lines of code...
    <p>
    <code>
        from numpy import *<br>
        exp(-2*pi)
    </code>
    </p>
    """
iframe = folium.element.IFrame(html=html, width=500, height=300)
popup = folium.Popup(iframe, max_width=2650)

folium.Marker([30,-100], popup=popup).add_to(swiss_map)

<font color='red'>**NOTE: In case you cannot see the map in notebook, please open the Maps/swiss-map.html file**</font>

# Bonus

We seperate cantons that belong to the French from those that belong to the German part of Switzerland. Some cantons are bilingual. FR and VS are assigned to the French part and BE is assigned to the German part. The rest of the cantons belong to the other part of Switzerland, where Italian or Romansh is spoken.

In [28]:
french_part = ['GE', 'VD', 'NE', 'JU', 'FR', 'VS']
german_part = ['ZH', 'BS', 'BL', 'AG', 'AI', 'AR', 'GL', 'LU', 'NW', 'OW', 'SH', 'SZ', 'SO','SG', 'TG', 'UR', 'BE']
uni_grants['Part'] = ''

We assign each university to the French, the German or the other part of Switzerland.

In [29]:
uni_grants['Part'] = uni_grants.apply(lambda row : split_switzerland(row['Canton'], french_part, german_part), axis=1)

We group by part and sum the approved amount. **The results are again reported in MCHF**

In [30]:
uni_grants_per_part = uni_grants.groupby(by='Part')['Approved Amount'].sum()
uni_grants_per_part

Part
french    5.040978e+09
german    6.630426e+09
other     1.371271e+07
Name: Approved Amount, dtype: float64

In [31]:
french_grants = uni_grants_per_part.loc['french']
german_grants = uni_grants_per_part.loc['german']
other_grants = uni_grants_per_part.loc['other']

In [32]:
uni_grants['Grant per Part'] = ''

In [33]:
uni_grants.set_value(uni_grants['Part'] == 'german', 'Grant per Part', german_grants/1000000)
uni_grants.set_value(uni_grants['Part'] == 'french', 'Grant per Part', french_grants/1000000)
uni_grants.set_value(uni_grants['Part'] == 'other', 'Grant per Part', other_grants/1000000)
uni_grants.head()

,Approved Amount,Canton,End Date,Project Number,Start Date,University,Part,Grant per Part
1,41022.0,GE,30.09.1976,4.0,01.10.1975,Université de Genève,french,5040.98
3,52627.0,BS,30.09.1976,6.0,01.10.1975,Universität Basel,german,6630.43
5,53009.0,FR,31.12.1978,8.0,01.01.1976,Université de Fribourg,french,5040.98
6,25403.0,FR,31.12.1976,9.0,01.01.1976,Université de Fribourg,french,5040.98
7,47100.0,ZH,31.03.1977,10.0,01.10.1975,Universität Zürich,german,6630.43


<font color='red'>**NOTE: The <code>legend_name</code> is not displayed in the map. The color palette is used to visualize the amount of approved grant per canton. The deeper the green hue, the higher the approved amount of money (MCHF).**</font>

In [34]:
canton_geo = r'ch-cantons.topojson.json'
canton_data = uni_grants

part_map = folium.Map(location=[46.762579, 7.927242], zoom_start=8)
part_map.choropleth(geo_path= canton_geo, data= canton_data,
                    key_on='feature.id',
                    threshold_scale=list(range(0, int(german_grants/1000000), 1200)),
                    columns=['Canton', 'Grant per Part'],
                    fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2, 
                    legend_name='Approved Grants per Part (MCHF)',
                    topojson='objects.cantons')

for key,value in uni_locs.items():
    folium.Marker(value, popup=key).add_to(part_map)

part_map.save('Maps/part-map.html')
part_map

We notice that there is a small difference in the amount of approved grants between the German and the French part of Switzerland.

<font color='red'>**NOTE: In case you cannot see the map in notebook, please open the Maps/part-map.html file**</font>

## Analysis of Approved Amount per Year per University

Now, we perform an analysis of the approved grant amount per year per university. This gives us an intuition of how grants evolve over time for each university. **All results are again in MCHF**.

In [35]:
uni_grants = uni_grants[['Approved Amount','Start Date', 'University', 'Canton']]
uni_grants.head(2)

,Approved Amount,Start Date,University,Canton
1,41022.0,01.10.1975,Université de Genève,GE
3,52627.0,01.10.1975,Universität Basel,BS


Here, we display the number of grants given.

In [36]:
uni_grants.shape[0]

43249

However, some records do not contain a start date and should be removed for our analysis.

In [37]:
uni_grants.dropna(subset=['Start Date'], inplace=True)
uni_grants.shape[0]

43239

We convert grant amounts to MCHF.

In [38]:
uni_grants['Approved Amount'] = uni_grants['Approved Amount']/1000000

We extract the year in which the grant was given.

In [39]:
uni_grants['Year'] = uni_grants['Start Date'].map(lambda row: row.split('.')[-1])
uni_grants.head()

,Approved Amount,Start Date,University,Canton,Year
1,0.041022,01.10.1975,Université de Genève,GE,1975
3,0.052627,01.10.1975,Universität Basel,BS,1975
5,0.053009,01.01.1976,Université de Fribourg,FR,1976
6,0.025403,01.01.1976,Université de Fribourg,FR,1976
7,0.047100,01.10.1975,Universität Zürich,ZH,1975


We group by university, year and canton to find the total approved amount per university per year.

In [40]:
grant_per_uni_per_year = uni_grants.groupby(by=['University','Year','Canton']).sum()
grant_per_uni_per_year.reset_index(inplace=True)
grant_per_uni_per_year.head(4)

,University,Year,Canton,Approved Amount
0,Berner Fachhochschule,1978,BE,0.006000
1,Berner Fachhochschule,1992,BE,0.364680
2,Berner Fachhochschule,2001,BE,0.434415
3,Berner Fachhochschule,2002,BE,0.019213


In [41]:
grant_per_uni_per_year['Year'] = grant_per_uni_per_year['Year'].map(lambda row: '01.01.' + row)
grant_per_uni_per_year['Year'] = pd.to_datetime(grant_per_uni_per_year['Year'])

In [42]:
grant_per_uni_per_year.set_index('Year', inplace=True)
grant_per_uni_per_year.head(2)

,University,Canton,Approved Amount
Year,,,
1978-01-01,Berner Fachhochschule,BE,0.00600
1992-01-01,Berner Fachhochschule,BE,0.36468


In [43]:
histo_per_uni = {}
for name,loc in uni_locs.items():
    uni_data = grant_per_uni_per_year[grant_per_uni_per_year['University'] == name]
    line = vincent.Line(uni_data['Approved Amount'])
    line.width = 400
    line.height = 200
    line.axis_titles(x='Date', y='Approved Amount (MCHF)')
    histo_per_uni[name] = line

We display a map with the location of the swiss universities.
* **click** on the red cycles: you will see the university names
* **click** on the blue markers: you will see how the approved grant amount for each university evolves per year

In [44]:
canton_geo = r'ch-cantons.topojson.json'
canton_data = grants_per_canton

swiss_map = folium.Map(location=[46.762579, 7.927242], zoom_start=8)
swiss_map.choropleth(geo_path= canton_geo, data=canton_data,
                     key_on='feature.id',
                     threshold_scale=list(range(int(min_amount), int(max_amount),700)),
                     columns=['Canton', 'Approved Amount'],
                     fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2, 
                     legend_name='Approved Amount (MCHF)',
                     topojson='objects.cantons')

for name,loc in uni_locs.items():
    popup = folium.Popup(max_width=800,).add_child(folium.Vega(histo_per_uni[name], width=500, height=250))
    folium.CircleMarker(location=loc, radius=100, popup=name, color='red', fill_color='red').add_to(swiss_map)
    folium.Marker(loc, popup=popup).add_to(swiss_map)

swiss_map.save('Maps/swiss-map-per-year.html')
swiss_map

<font color='red'>**NOTE: In case you cannot see the map in notebook, please open the Maps/swiss-map-per-year.html file**</font>